In [1]:
# lets download the annotations from http://mscoco.org/dataset/#download
import os
import json
coco = json.load(open('../../Data/neural_talk_data/caption_datasets/dataset_coco.json', 'r'))

In [6]:
name = "H7_18_2_augment_42"
idx = name.find('augment')
ident = name[0:idx-1]
print(ident)
print(name.find('augment'))

naked = "H7_18_2"
print(naked.find('augment'))

H7_18_2
8
-1


In [29]:
from __future__ import print_function
import numpy as np
from copy import copy
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
    
def cumprod(inputs, dim = 1, exclusive=True):
    # https://github.com/tensorflow/tensorflow/blob/master/tensorflow/g3doc/api_docs/python/functions_and_classes/shard0/tf.cumprod.md

    if type(inputs) is not Variable:
        temp = torch.cumprod(inputs, dim)
        if not exclusive:
            return temp
        else:
            temp =  temp / (inputs[0].expand_as(temp) + 1e-8)
            temp[-1] = temp[-1]/(inputs[-1]+1e-8)
            return temp
    else:
        shape_ = inputs.size()
        ndim = len(shape_)
        n_slot = shape_[dim]
        output = Variable(inputs.data.new(*shape_).fill_(1.0), requires_grad = True)
        slice_ = [slice(0,None,1) for _ in range(ndim)]
        results = [[]] * n_slot
            
        for ind in range(0, n_slot):   
            this_slice, last_slice = copy(slice_), copy(slice_)
            this_slice[dim] = ind
            last_slice[dim] = ind-1      
            this_slice = tuple(this_slice)
            last_slice = tuple(last_slice)
            if exclusive: 
                if ind > 0:   
                    results[ind]  = results[ind-1]*inputs[last_slice]
                else:
                    results[ind] =  torch.div(inputs[this_slice], inputs[this_slice]+1e-8)
            else:    
                if ind > 0:   
                    results[ind]  = results[ind - 1]*inputs[this_slice]
                else:
                    results[ind] =  inputs[this_slice]
        
        return torch.stack(results, dim)



W = nn.Parameter(torch.Tensor(4,7,10))

data_pre = data_var * W
#print(data)
#result = cumprod(data, 1, True)
result_var = cumprod(data_pre)

#print(result_var)

loss = torch.sum(result_var) *0.5
loss.backward()

print(W.grad)

In [30]:
import torch.nn as nn

data = torch.rand(4,7,10)

data_var = Variable(data)

result = cumprod(data, 1, True)
result_var = cumprod(data_var)

print(result, result_var)




(0 ,.,.) = 

Columns 0 to 5 
   1.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00
  2.7183e-01  5.1821e-01  3.1315e-01  6.0220e-01  2.1070e-01  5.1511e-02
  1.2155e-02  4.0495e-02  9.5343e-02  5.3563e-02  9.2726e-02  1.1480e-02
  1.3138e-03  2.0646e-02  3.0269e-02  1.9387e-03  8.6944e-02  1.8767e-03
  2.4156e-04  1.3680e-02  2.6423e-02  7.9429e-04  1.9606e-02  8.3287e-05
  8.9345e-05  2.5854e-03  1.2000e-02  3.6397e-04  1.4546e-02  6.3052e-05
  8.0057e-05  1.0126e-03  2.1515e-04  1.2945e-04  4.1774e-03  3.5443e-06

Columns 6 to 9 
   1.0000e+00  1.0000e+00  1.0000e+00  1.0000e+00
  3.2066e-01  7.5134e-01  2.1201e-01  8.7637e-01
  2.3872e-02  1.0548e-01  1.1280e-01  7.0206e-02
  1.3599e-02  3.7222e-02  1.7532e-02  2.9286e-02
  1.0159e-02  2.0240e-02  1.0189e-02  2.6916e-02
  5.5381e-03  1.0313e-02  1.0006e-02  1.0428e-02
  5.0010e-03  4.2063e-03  7.3943e-03  8.1591e-03

(1 ,.,.) = 

Columns 0 to 5 
   3.1102e+00  1.7116e+00  6.2718e-01  6.4754e-01  3.7530e+00  8.79

Variable containing:
(0 ,.,.) = 

Columns 0 to 5 
   1.3591e-01  2.5911e+07  1.5657e-01  3.0110e+07  1.0535e+07  2.5756e+06
 -1.6945e+08  0.0000e+00 -1.3292e+09  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00

Columns 6 to 9 
   1.6033e+07  3.7567e+07  1.0600e+07  4.3818e+07
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00

(1 ,.,.) = 

Columns 0 to 5 
   4.2271e+07  4.4348e+07  9.8200e+06  1.9497e+

In [99]:
dat = torch.rand(4,3,2)
row, col, dim = dat.size()

print(dat)
flat_dat = dat.view(-1,2)
print(flat_dat)
first = torch.LongTensor([0,2,1])
second = torch.LongTensor([1,0,2])

print(dat[0,1])
index = first*col + second
select = flat_dat[index]

print('=================')
print(select)


(0 ,.,.) = 
  0.7615  0.5426
  0.0270  0.2162
  0.4347  0.5452

(1 ,.,.) = 
  0.4225  0.8997
  0.9161  0.7573
  0.6608  0.1834

(2 ,.,.) = 
  0.6764  0.6198
  0.0159  0.6918
  0.7290  0.8699

(3 ,.,.) = 
  0.7890  0.4502
  0.1355  0.5624
  0.8061  0.9127
[torch.FloatTensor of size 4x3x2]


 0.7615  0.5426
 0.0270  0.2162
 0.4347  0.5452
 0.4225  0.8997
 0.9161  0.7573
 0.6608  0.1834
 0.6764  0.6198
 0.0159  0.6918
 0.7290  0.8699
 0.7890  0.4502
 0.1355  0.5624
 0.8061  0.9127
[torch.FloatTensor of size 12x2]


 0.0270
 0.2162
[torch.FloatTensor of size 2]


 0.0270  0.2162
 0.6764  0.6198
 0.6608  0.1834
[torch.FloatTensor of size 3x2]



In [2]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack

def sort_batch(data, seq_len):
    batch_size = data.size(0)
    sorted_seq_len, sorted_idx  = torch.sort(seq_len, dim=0, descending=True)
    sorted_data = data[sorted_idx]
    _, reverse_idx  = torch.sort(sorted_idx, dim=0, descending=False)
    return sorted_data, sorted_seq_len, reverse_idx

data = torch.rand(4,7,10)
lens = torch.LongTensor([3,7,2,1])

s_data, s_len, reverse_idx = sort_batch(data, lens)

emb = pack(Variable(s_data), list(s_len),batch_first=True)

rnn = nn.LSTM(10, 20, 2, batch_first =True, bidirectional=False)
input = emb
h0 = Variable(torch.randn(2, 6, 20))
c0 = Variable(torch.randn(2, 6, 20))
packed_output, hn = rnn(input)
result, lens = unpack(packed_output)
print(result.size())


torch.Size([7, 4, 20])


In [7]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack

data = Variable(torch.rand(4,7,10))
lens = torch.LongTensor([3,7,2,1])

#s_data, s_len, reverse_idx = sort_batch(data, lens)

rnn = nn.LSTM(10, 20, 2, batch_first =True, bidirectional=False)

h0 = Variable(torch.randn(2, 6, 20))
c0 = Variable(torch.randn(2, 6, 20))
output, hn = rnn(data)

#result, lens = unpack(packed_output)
print(output.size())


torch.Size([4, 7, 20])


In [4]:
import torch
from torch.autograd import Variable

a = Variable(torch.Tensor([1]))

cuda = torch.cuda.is_available()
print(cuda)

True


In [112]:

def two_ind(data, first, second):
    '''
    data: 3 dimension
    first: 1 d tensor
    second: 1 d tensor
    '''
    row, col, dim = data.size()
    index = first*col + second
    last_out = result.view(-1, dim)[index]
    return last_out
    
lens_th = Variable(torch.LongTensor(lens))    
first  = (lens_th -1).data 
second = torch.range(0, len(lens)-1).long()

print(first, second)
last_out = two_ind(result, first, second)
#last_out = torch.index_select(torch.index_select(second, 0, lens_th-1), 1, second)
#last_out = result[lens_th.data-1, :][:,second]
                   
print('result', result.size())
print('last out',last_out.size())

print(last_out[:,0:8])
print('---------------')
print(result[:, :, 0:8])

(
 6
 2
 1
 0
[torch.LongTensor of size 4]
, 
 0
 1
 2
 3
[torch.LongTensor of size 4]
)
('result', torch.Size([7, 4, 20]))
('last out', torch.Size([4, 20]))
Variable containing:
-0.0213  0.0501 -0.1353 -0.0388 -0.0474 -0.2192  0.0512 -0.1225
-0.0184  0.0430 -0.1112 -0.0204 -0.0506 -0.1931  0.0632 -0.0996
-0.0140  0.0469 -0.0929 -0.0135 -0.0487 -0.1604  0.0630 -0.0850
-0.0202  0.0292 -0.0607  0.0021 -0.0313 -0.1138  0.0473 -0.0582
[torch.FloatTensor of size 4x8]

---------------
Variable containing:
(0 ,.,.) = 
 -0.0203  0.0331 -0.0582  0.0013 -0.0381 -0.1165  0.0493 -0.0546
 -0.0086  0.0298 -0.0589 -0.0036 -0.0282 -0.1145  0.0368 -0.0562
 -0.0119  0.0327 -0.0614 -0.0033 -0.0302 -0.1074  0.0475 -0.0600
 -0.0202  0.0292 -0.0607  0.0021 -0.0313 -0.1138  0.0473 -0.0582

(1 ,.,.) = 
 -0.0281  0.0402 -0.0977 -0.0021 -0.0458 -0.1637  0.0722 -0.0916
 -0.0160  0.0365 -0.0948 -0.0093 -0.0365 -0.1666  0.0549 -0.0874
 -0.0140  0.0469 -0.0929 -0.0135 -0.0487 -0.1604  0.0630 -0.0850
  0.0000  0.000

In [120]:
batch1 = torch.randn(5, 4)
batch2 = torch.randn(5, 4)

print(batch1,batch2)

def cosine_similarity(first, second):
    """
    compute the cosine similarity between keys to each of the 
    memory slot.

    Parameters:
    ----------
    first: Tensor (batch_size, dim)
        the memory matrix to lookup in
    second: Tensor (batch_size, dim)
        the keys to query the memory with
    Returns: Tensor (batch_size, 1)
    """
    first_norm  = torch.norm(first,  2, 1)
    second_norm = torch.norm(second, 2, 1)

    normalized_first  = torch.div(first,  first_norm.expand_as(first) + 1e-10)
    normalized_second = torch.div(second, second_norm.expand_as(second) + 1e-10)

    return torch.sum(normalized_first*normalized_second,1)

cos = cosine_similarity(batch1, batch2)
print(cos)

(
-1.5788  1.3199  1.7161  0.5836
 0.9811  2.5551 -0.3949  0.9135
 0.0840 -0.0228  1.3977 -1.0893
 0.4723 -0.3251 -0.4061  1.2842
-1.5411 -1.1587  0.8768  0.8281
[torch.FloatTensor of size 5x4]
, 
-1.0217  0.7216  0.3534 -0.9644
-0.1993 -0.1303  0.6186  0.4552
 0.2413  0.9452 -1.2923 -0.1830
 0.4121  1.4143  0.4108  0.6924
 0.6960  1.5349  1.8151  0.0185
[torch.FloatTensor of size 5x4]
)

 0.5879
-0.1524
-0.5563
 0.1860
-0.2209
[torch.FloatTensor of size 5x1]



In [79]:
import numpy as np
data = np.random.rand(4,5,6)
first = np.array([0,1,2])
second = np.array([2,3,1])

res = data[first, second]

print(res)

[[ 0.48210773  0.752584    0.64944821  0.19314062  0.03171795  0.26378825]
 [ 0.14147113  0.4699371   0.66200108  0.78925308  0.94670398  0.18808383]
 [ 0.59918571  0.57468536  0.66673667  0.98518522  0.20246986  0.76444525]]


In [2]:
import h5py
import numpy as np

def create_dataset(h5py_file, string_list):
    data = np.array([['I', 'am', 'a', 'sentence'], ['another', 'sentence']], dtype=object)
    string_dt = h5py.special_dtype(vlen=str)
    h5py_file.create_dataset("sentences", data=data, dtype=string_dt)
    
h5File.close()
h5File=h5py.File('xxx.h5','w')

create_dataset(h5File)
h5File.close()


NameError: name 'h5File' is not defined

In [16]:
import numpy as np
aa =  np.eye(5)
print aa[np.array([1,3,1])]

[[ 0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.]]


# Generator annotation json (may be done already)

In [12]:
import sys
import json
import os
import shutil
import numpy as np
import h5py

sys.path.insert(0, '..')
from proj_utils.local_utils import *

Data_root = os.path.join('..','..','..', 'Data', 'bladder_data')
save_root = os.path.join('..','..','..', 'Data', 'bladder_data')

annotation_folder = os.path.join(Data_root, 'Annotation')
Img_folder = os.path.join(Data_root, 'Img') 


hdf5_filepath = os.path.join(save_root, 'images_caption.h5')
hdf5_ref_filepath = os.path.join(save_root, 'images_caption_ref.json')

In [ ]:
def get_identifier(name):
    idx = name.find('augment')
    if idx == -1:
        return name
    else:
        ident = name[0:idx - 1]
        return ident
with h5py.File(hdf5_filepath,'a') as ImgHdf5:
    

In [17]:
output_json = os.path.join(save_root,'total_anno.json') 

anno_filelist, anno_filenames = getfilelist(annotation_folder,['.json'] )

total_num = len(anno_filelist)
print(total_num)

total_json = []
test_num = total_num

# pool for one hot label
n_labels = 4
one_hot = np.eye(n_labels).astype(np.int)
def get_cnn_img(thisimg, shape, norm=False):
    ''' 
    Parameters:
    -----------
    shape, (row, col, channel).
    norm: if you preder to normalize iter

    Returns:
    ----------
    img: Tensor of shape(channel, row, col)

    '''
    if isinstance(thisimg, np.ndarray):
        img = thisimg
    elif type(thisimg) is str:
        img = imread(thisimg)
    img  = pre_process_img(img, yuv = False, norm= norm)  #(img, yuv = False,norm= local_norm)

    img =  imresize_shape(img,shape)
    # transpose the img to order (channel, row, col)
    img = np.transpose(img, (2,0,1))
    return img

def parse_anno(anno_dict_list):

    str_list = []
    if type(anno_dict_list) is not list:
        anno_dict_list = [anno_dict_list]
    for anno_dict in anno_dict_list:         
        thisstr = ''
        for k, v in anno_dict.items():
            thisstr = thisstr + ' '+ k.title() +': '
            thisstr  = thisstr + ' ' + v[1]
        str_list.append(thisstr)
    return str_list


with h5py.File(hdf5_filepath,'w') as ImgHdf5:
    #dset_img = ImgHdf5.create_dataset("images", (total_num, 3, 224, 224), dtype='uint8')
    dset_img = ImgHdf5.create_dataset("images", (total_num, 3, 224, 224), dtype='uint8')

    dset_conclusion = ImgHdf5.create_dataset("conclusion", (total_num, n_labels), dtype='int')

    string_dt = h5py.special_dtype(vlen=str)

    sentence_list  = []
    filename_list = []
    filename_dict = {}

    for idx, (fp, fn) in enumerate(zip(anno_filelist, anno_filenames )): 
        if idx == test_num:
            break

        img_name = fn + '.png'
        thisimg = imread(os.path.join(Img_folder, img_name))
        filename_dict[fn] = idx

        new_img  = get_cnn_img(thisimg, (224, 224,3), norm=False).astype(np.uint8)

        with open(fp) as data_file:    
            thisjson = json.load(data_file)
        sentence_list.append(parse_anno(thisjson)) 
        filename_list.append(fn)

        this_conclusion = thisjson[0]['conclusion'][0] #not beautiful
        dset_img[idx] = new_img
        dset_conclusion[idx] = one_hot[this_conclusion]


    sentence_data = np.array(sentence_list, dtype=object)
    ImgHdf5.create_dataset("sentences", data=sentence_data, dtype=string_dt)

    filename_data = np.array(filename_list, dtype= object)
    ImgHdf5.create_dataset("filenames", data=filename_data, dtype=string_dt)
    ImgHdf5.close()

    with open(hdf5_ref_filepath,'w') as f:
        json.dump(filename_dict, f)


50008


In [ ]:
with h5py.File(hdf5_filepath,'w') as ImgHdf5:

# Load and test the hdf5 file

In [6]:
import h5py

import sys
import json
import os
import shutil
import numpy as np
import h5py

sys.path.insert(0, '..')
#from utils.local_utils import *

Data_root = os.path.join('..','..', 'Data', 'bladder_data')
annotation_folder = os.path.join(Data_root, 'Annotation') 
Img_folder = os.path.join(Data_root, 'Img') 

output_json = os.path.join(Data_root,'total_anno.json') 

# anno_filelist, anno_filenames = getfilelist(annotation_folder,['.json'] )

# total_num = len(anno_filelist)
# print(total_num)

# total_json = []
# test_num = total_num

# # pool for one hot label
# n_labels = 4
# one_hot = np.eye(n_labels).astype(np.int)


hdf5_filepath = os.path.join(Data_root, 'images_caption.h5')


with h5py.File(hdf5_filepath,'r') as testh5:
    imgs = testh5['images']
    sents = testh5['sentences']
    labs  = testh5['conclusion']
    fns   = testh5['filenames']
    
    print(imgs.shape)
    print(sents.shape)
    sents_list = sents.value[[1,3,1,4]]
    this_sent = sents_list[0]
    print(this_sent)
    print('-----------------')
    print(this_sent[0])
    
    



(50008, 3, 224, 224)
(50008, 5)
[ ' Polarity:  Polarity is negligibly lost. Nucleoli:  The nucleoli of nuclei are inconspicuous. Nuclear_Feature:  The nuclei are moderately pleomorphic. Mitosis:  There are rarely mitotic figures throughout the tissue. Nuclear_Crowding:  Nuclei are moderately crowded together. Conclusion:  Lg / punlmp.'
 ' Polarity:  Basement membrane polarity is negligibly lost. Nucleoli:  The nucleoli of nuclei are inconspicuous. Nuclear_Feature:  Moderate pleomorphism of the nuclei can be seen. Mitosis:  Mitosis is rare. Nuclear_Crowding:  Moderate nuclear crowding is present. Conclusion:  Lg / punlmp.'
 ' Polarity:  Polarity of nuclei is negligibly lost. Nucleoli:  The nucleoli are mostly inconspicuous. Nuclear_Feature:  Moderate pleomorphism is present in the nuclei. Mitosis:  Mitotic figures are rare. Nuclear_Crowding:  The nuclei are moderately crowded. Conclusion:  Lg / punlmp.'
 ' Polarity:  Polarity is negligibly lost. Nucleoli:  The nuclei have inconspicuous 

# Make dictionary

In [16]:
from collections import OrderedDict
import json
import re

def split_words(words):
    return re.findall(r'\w+|\S+', words)

save_root = os.path.join('..','..', 'Data', 'bladder_data')

dict_path = os.path.join(save_root,'dictionary.json')
worddict = OrderedDict()
worddict['<eos>'] = 0
worddict['UNK'] = 1
wordIndex = 2

Dicts = {}


with h5py.File(hdf5_filepath,'r') as testh5:
    imgs = testh5['images']
    sents = testh5['sentences']
    labs  = testh5['conclusion']
    fns   = testh5['filenames']
    all_sents = sents.value
    for this_list in all_sents:
        for this_sent in this_list:
            this_sent = this_sent.decode()
            words = split_words(this_sent)
            for k in words:
                if k not in worddict:
                    worddict[k] = wordIndex
                    wordIndex = wordIndex + 1

word_idict = dict()
for kk, vv in worddict.items():
    word_idict[vv] = kk
word_idict[0] = '<eos>'
word_idict[1] = 'UNK'

Dicts['encoder'] = worddict
Dicts['decoder'] = word_idict
with open(dict_path,'w') as f:
    json.dump(Dicts, f)

        

In [3]:
import torch
import numpy as np

from torch.autograd import Variable
def to_variable(x, requires_grad=False, cuda=False):
    if type(x) is Variable:
        return x
    if type(x) is np.ndarray:
        x = torch.from_numpy(x)
    if cuda:
       return Variable(x, requires_grad=requires_grad).cuda()
    else:
       return Variable(x, requires_grad=requires_grad) 

def to_device(src, ref=None, var = True):
    
    src = to_variable(src) if var else src
    return src.cuda(ref.get_device()) if (ref is not None and ref.is_cuda) else src

def one_hot(nwords, ref=None):
    eye_tensor = to_device(torch.eye(nwords),ref, var=False)
    def f(tensor_int):
        if type(tensor_int) is int:
            tensor_int  = [tensor_int]
        if type(tensor_int) is list :
            tensor_int = torch.LongTensor(tensor_int)
        tensor_int = to_device(tensor_int, ref, var=False)
        size = list(tensor_int.size()) 
        flat = tensor_int.view(-1)
        flat_emb = torch.index_select(eye_tensor, 0, flat)
        new_shape = size + [nwords]
        emb = flat_emb.view(*new_shape)
        return Variable(emb)
    return f

Input = torch.Tensor([ [1,1],[3,1],[3,2],[2,2]]).long()
ref = Input.cuda()
print(Input)

emb = one_hot(4, ref)
res = emb([0,1,3,1,2])
print(res.data)


 1  1
 3  1
 3  2
 2  2
[torch.LongTensor of size 4x2]


 1  0  0  0
 0  1  0  0
 0  0  0  1
 0  1  0  0
 0  0  1  0
[torch.cuda.FloatTensor of size 5x4 (GPU 0)]



In [ ]:

np.mod(batch_count, args.validfreq) == 0






print coco.keys()
print type(coco['images'])
print type(coco['dataset'])

In [11]:
print coco['images'][0]
print('---------------')
print coco['dataset']
print('---------------')
print(coco['images'][0].keys())
print('---------------')
print coco['images'][0]['sentences']
print('---------------')
print coco['images'][0]['split']

{u'sentids': [770337, 771687, 772707, 776154, 781998], u'filepath': u'val2014', u'filename': u'COCO_val2014_000000391895.jpg', u'imgid': 0, u'split': u'test', u'sentences': [{u'tokens': [u'a', u'man', u'with', u'a', u'red', u'helmet', u'on', u'a', u'small', u'moped', u'on', u'a', u'dirt', u'road'], u'raw': u'A man with a red helmet on a small moped on a dirt road. ', u'imgid': 0, u'sentid': 770337}, {u'tokens': [u'man', u'riding', u'a', u'motor', u'bike', u'on', u'a', u'dirt', u'road', u'on', u'the', u'countryside'], u'raw': u'Man riding a motor bike on a dirt road on the countryside.', u'imgid': 0, u'sentid': 771687}, {u'tokens': [u'a', u'man', u'riding', u'on', u'the', u'back', u'of', u'a', u'motorcycle'], u'raw': u'A man riding on the back of a motorcycle.', u'imgid': 0, u'sentid': 772707}, {u'tokens': [u'a', u'dirt', u'path', u'with', u'a', u'young', u'person', u'on', u'a', u'motor', u'bike', u'rests', u'to', u'the', u'foreground', u'of', u'a', u'verdant', u'area', u'with', u'a', u

In [ ]:
bladder = json.load(open('../../Data/bladder/caption_datasets/dataset_coco.json', 'r'))